In [5]:
import polars as pl
import pysentiment2 as ps
import math
import os
import re
import urllib.parse

In [27]:
df_dict = {}

for company_name in os.listdir(r"C:\Users\310\Desktop\Progects_Py\prepared_data\full_snp_five_hundred"):
    
    file_path = os.path.join(r"C:\Users\310\Desktop\Progects_Py\prepared_data\full_snp_five_hundred", company_name)

    match = re.search(r"\\([A-Z]+(?:\.[A-Z])?)_reports\.parquet", file_path)

    if match:
        ticker_symbol = match.group(1)
        print("Ticker Symbol:", ticker_symbol, '\n', file_path)
    else:
        print(f"Ticker symbol for {company_name} is not found")

    df_name = f"{ticker_symbol}"
    df = pl.read_parquet(file_path)

    df_dict[df_name] = df

Ticker Symbol: AEE 
 C:\Users\310\Desktop\Progects_Py\prepared_data\full_snp_five_hundred\AEE_reports.parquet
Ticker Symbol: AKAM 
 C:\Users\310\Desktop\Progects_Py\prepared_data\full_snp_five_hundred\AKAM_reports.parquet
Ticker Symbol: ALGN 
 C:\Users\310\Desktop\Progects_Py\prepared_data\full_snp_five_hundred\ALGN_reports.parquet
Ticker Symbol: AME 
 C:\Users\310\Desktop\Progects_Py\prepared_data\full_snp_five_hundred\AME_reports.parquet
Ticker Symbol: AMT 
 C:\Users\310\Desktop\Progects_Py\prepared_data\full_snp_five_hundred\AMT_reports.parquet
Ticker Symbol: AMZN 
 C:\Users\310\Desktop\Progects_Py\prepared_data\full_snp_five_hundred\AMZN_reports.parquet
Ticker Symbol: ANSS 
 C:\Users\310\Desktop\Progects_Py\prepared_data\full_snp_five_hundred\ANSS_reports.parquet
Ticker Symbol: ARE 
 C:\Users\310\Desktop\Progects_Py\prepared_data\full_snp_five_hundred\ARE_reports.parquet
Ticker Symbol: AXON 
 C:\Users\310\Desktop\Progects_Py\prepared_data\full_snp_five_hundred\AXON_reports.parquet


KeyboardInterrupt: 

In [7]:
total_num_of_rep = 0

for value in df_dict.values():
    total_num_of_rep += value.width
print(total_num_of_rep)

7807


In [8]:
total_num_of_words = 0

for value in df_dict.values():
    for col in value:
        total_num_of_words += col.count()

print(total_num_of_words)

208814303


In [9]:
#average doc length
average_word_count = total_num_of_words/total_num_of_rep
print(f'Average doc length in words: {round(average_word_count)}')
ln_average_word_count = math.log(average_word_count)

Average doc length in words: 26747


In [10]:
#here I have checked if there ara weird numder of words for reports
for company_name, df in df_dict.items():
    for col in df:
        if col.count() < 5000:
            print(f"{company_name}, {col.name}: {col.count()}")
        elif col.count() > 100000:
            print(f"{company_name}, {col.name}: {col.count()}")

BK, 2019-02-27: 4573
BK, 2020-02-27: 4046
BK, 2022-02-25: 4911
BX, 2019-03-01: 108861
BX, 2020-02-28: 101887
BX, 2021-02-26: 105295
BX, 2022-02-25: 104668
BX, 2023-02-24: 104516
CVS, 2023-02-08: 102008
C, 2021-02-26: 127501
C, 2022-02-28: 128374
C, 2023-02-27: 131435
DUK, 2021-02-25: 100697
EMR, 2020-02-05: 4067
EMR, 2021-02-03: 4342
ETR, 2019-02-26: 168747
ETR, 2020-02-21: 160811
ETR, 2021-02-26: 195291
ETR, 2022-02-25: 199557
ETR, 2023-02-24: 223235
EXC, 2019-02-08: 133056
EXC, 2021-02-24: 117172
EXC, 2022-02-25: 105179
FITB, 2021-02-26: 114400
FITB, 2022-02-25: 108102
FITB, 2023-02-24: 101084
GS, 2021-02-22: 106729
GS, 2022-02-25: 109072
GS, 2023-02-24: 131119
HIG, 2021-02-19: 110565
HIG, 2022-02-18: 105016
JPM, 2019-02-26: 104752
JPM, 2021-02-23: 136588
JPM, 2022-02-22: 125388
JPM, 2023-02-21: 124644
MET, 2019-02-22: 134423
MET, 2020-02-21: 116417
MET, 2021-02-19: 130346
MET, 2022-02-18: 126700
MET, 2023-02-23: 123575
PCG, 2020-02-18: 114788
PCG, 2021-02-25: 112262
PNC, 2021-02-26:

In [15]:
lm = ps.LM()
hiv4 = ps.HIV4() 

def compute_pysentiment(series):

    score_lst = []

    doc_string = series.str.concat(" ", ignore_nulls=True)

    tokens_lm = lm.tokenize(doc_string[0])
    tokens_hiv4 = hiv4.tokenize(doc_string[0])

    score_lm = lm.get_score(tokens_lm)
    score_hiv4 = hiv4.get_score(tokens_hiv4)

    raw_lm = score_lm['Positive'] - score_lm['Negative']
    raw_hiv4 = score_hiv4['Positive'] - score_hiv4['Negative']

    score_lst.append(score_lm['Polarity'])
    score_lst.append(raw_lm)

    score_lst.append(score_hiv4['Polarity'])
    score_lst.append(raw_hiv4)

    score_lst.append(series.count())
  
    return score_lst

In [16]:
#the resulting list contain 1: lm_polarity score; 2: lm_raw score; 3: hiv4_polarity score; 4: hiv4_raw score
compute_pysentiment(example_series)

[-0.44518272388273855, -536, 0.3756441550377966, 1385, 23700]

In [13]:
#example df and series 
example_series = df_dict['ALGN'][df_dict['ALGN'].columns[1]]
example_df = df_dict['ALGN']
example_df


2019-02-28,2019-05-02,2019-08-01,2019-10-31,2020-02-28,2020-05-05,2020-07-31,2020-10-30,2021-02-26,2021-05-05,2021-08-04,2021-11-02,2022-02-25,2022-05-05,2022-08-04,2022-11-04,2023-02-27,2023-05-05,2023-08-04,2023-11-03
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""UNITED""","""UNITED""","""UNITED""","""UNITED""","""UNITED""","""UNITED""","""UNITED""","""UNITED""","""UNITED""","""UNITED""","""UNITED""","""UNITED""","""UNITED""","""UNITED""","""UNITED""","""UNITED""","""UNITED""","""UNITED""","""UNITED""","""UNITED"""
"""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…"
"""AND""","""AND""","""AND""","""AND""","""AND""","""AND""","""AND""","""AND""","""AND""","""AND""","""AND""","""AND""","""AND""","""AND""","""AND""","""AND""","""AND""","""AND""","""AND""","""AND"""
"""EXCHANGE""","""EXCHANGE""","""EXCHANGE""","""EXCHANGE""","""EXCHANGE""","""EXCHANGE""","""EXCHANGE""","""EXCHANGE""","""EXCHANGE""","""EXCHANGE""","""EXCHANGE""","""EXCHANGE""","""EXCHANGE""","""EXCHANGE""","""EXCHANGE""","""EXCHANGE""","""EXCHANGE""","""EXCHANGE""","""EXCHANGE""","""EXCHANGE"""
"""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
null,null,null,null,null,null,null,null,"""capacities""",null,null,null,null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,"""and""",null,null,null,null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,"""on""",null,null,null,null,null,null,null,null,null,null,null


In [17]:
for company_name, df in df_dict.items():

    score_dict = {}

    for col in df:
        
        score_lst = compute_pysentiment(col)

        score_dict[col.name] = score_lst
    
    df = pl.DataFrame(score_dict)
    
    print(f'Iteration for {company_name} is done successfully')

 # Determine the output directory and file name
    output_dir = os.path.join('data', 'snp_500_scores')
    os.makedirs(output_dir, exist_ok=True)

    file_name_new = f"{company_name}.parquet"
    full_path = os.path.join(output_dir, file_name_new)
    full_path = os.path.normpath(full_path)

    print(f"Attempting to write to: {full_path}")

    # Write the DataFrame to Parquet
    df.write_parquet(full_path)

    print(f"{full_path} created successfully")

Iteration for AEE is done successfully
Attempting to write to: data\snp_500_scores\AEE.parquet
data\snp_500_scores\AEE.parquet created successfully
Iteration for AKAM is done successfully
Attempting to write to: data\snp_500_scores\AKAM.parquet
data\snp_500_scores\AKAM.parquet created successfully
Iteration for ALGN is done successfully
Attempting to write to: data\snp_500_scores\ALGN.parquet
data\snp_500_scores\ALGN.parquet created successfully
Iteration for AME is done successfully
Attempting to write to: data\snp_500_scores\AME.parquet
data\snp_500_scores\AME.parquet created successfully
Iteration for AMT is done successfully
Attempting to write to: data\snp_500_scores\AMT.parquet
data\snp_500_scores\AMT.parquet created successfully
Iteration for AMZN is done successfully
Attempting to write to: data\snp_500_scores\AMZN.parquet
data\snp_500_scores\AMZN.parquet created successfully
Iteration for ANSS is done successfully
Attempting to write to: data\snp_500_scores\ANSS.parquet
data\s

In [26]:
df_dict["BF.B"].get_column(df_dict["BF.B"].columns[0]).str.concat(" ", ignore_nulls=True).item()

'United StatesSecurities and Exchange CommissionWashington DC One For the quarterly period ended January ORFor the transition period from to Commission File No CorporationExact name of Registrant as specified in its telephone number including area code NAFormer name former address and former fiscal year if changed since last reportIndicate by check mark whether the registrant has filed all reports required to be filed by Section or of the Securities Exchange Act of during the preceding months or for such shorter period that the registrant was required to file such reports and has been subject to such filing requirements for the past days Yes th No oIndicate by check mark whether the registrant has submitted electronically every Interactive Data File required to be submitted pursuant to Rule of Regulation ST during the preceding months or for such shorter period that the registrant was required to submit such files Yes th No oIndicate by check mark whether the registrant is a large acce